# **Introduction**
TensorRT là một thư viện được phát triển bởi NVIDIA và là một phần của NVIDIA Deep Learning Accelerator (NVIDIA), dùng để tối ưu hóa mô hình AI, học máy chạy trên GPU. Giúp cải thiện tốc độ và hiệu suất khi triển khai môi hình trên các hệ thống nhúng và máy tính.

Bài học này sẽ trình bày cách sủ dụng TensorRT cơ bản cho Post-Training Quantization trên mô hình ResNet18.

Quy trình tối ưu hóa với TensorRT:
* (1) Chuyển đổi (Conversion) mô hình PyTorch thành dạng trung gian ONNX thông qua hàm torch.onnx.export để tương thích định dạng hỗ trợ của Tensort.

* (2) Xây dựng Engine (Building), tại đây TensorRT Builder phân tích đồ thị ONNX và tái cấu trúc mạng lưới, áp dụng các kỹ thuật lượng tử hóa như FP16 hoặc INT8 đi kèm Calibration để tạo ra một file Engine nhị phân (.trt) được tinh chỉnh đặc biệt cho phần cứng GPU đích.

* (3) Thực thi (Inference Runtime), file Engine được giải nén vào bộ nhớ, kết hợp với việc quản lý tài nguyên thủ công qua CUDA để vận hành luồng xử lý khép kín: sao chép dữ liệu từ Host xuống Device, thực hiện tính toán song song tốc độ cao, và chuyển kết quả về Host.

Lưu ý: Có nhiều cách cài đặt và sử dụng TensorRT (Thông qua API hoặc thư viện `torch-tensorrt` để cài đặt dễ hơn). Trong bài học này chỉ trình bày cách đơn giản nhất để chạy được trên Colab.

# **1. Cài đặt thư viện và chuẩn bị dữ liệu**

Đầu tiên, chúng ta cần cài đặt TensorRT (phiên bản tương thích với CUDA 12 trên Colab), PyCUDA để giao tiếp với GPU driver, và các thư viện hỗ trợ ONNX. Đồng thời tải file trọng số ResNet18 đã fine-tune trên CIFAR-10.

In [ ]:
!pip install tensorrt==10.0.1 --extra-index-url https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 GB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.9/879.9 kB 214.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 249.0 MB/s eta 0:00:00
  Created wheel for tensorrt: filename=tensorrt-10.0.1-py2.py3-none-any.whl size=16331 sha256=c6dfa72394251b5e3eb0522bffa954674c9f76fb77a40ec402758e89e7149862
  Stored in directory: /root/.cache/pip/wheels/ed/50/66/94cfda7829c4e8e71565109dc19bab66be78a78c5915ade5e5
  Created wheel for tensorrt-cu12: filename=tensorrt_cu12-10.14.1.48.post1-py2.py3-none-any.whl size=18166 sha256=f02882a98f619facd7d2f828c0823e0639d5163ebd81b5165c70dbc155880925
  Stored in directory: /root/.cache/pip/wheels/62/45/d8/cc48db84eae5237f52611a1a665d77130385ccb4cd1d0d8ae6
Successfully built tensorrt tensorrt-cu12
  Attempting un

Cài đặt các thư viện hỗ  trợ

In [ ]:
!pip install pycuda onnx onnxruntime onnxscript

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.4/693.4 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 11.5 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1.2-cp312-cp312-linux_x86_64.whl size=659050 sha256=635bfdd1ab8e3bfef47f7b9a115946a3fa4ed0c8f454b27

Tải file trọng số ResNet18 đã train trên CIFAR 10

In [ ]:
!gdown 1GrQM-axOHvnBcgegvVC2V7z1aaIHb4WS

Downloading...
From (original): https://drive.google.com/uc?id=1GrQM-axOHvnBcgegvVC2V7z1aaIHb4WS
From (redirected): https://drive.google.com/uc?id=1GrQM-axOHvnBcgegvVC2V7z1aaIHb4WS&confirm=t&uuid=cb45c7e6-10dd-4bb0-9b35-c0b4b7d06c7c
To: /content/resnet18_cifar10_finetuned.pth
100% 44.8M/44.8M [00:01<00:00, 35.4MB/s]


Import các thư viện cần thiết. Lưu ý tensorrt và pycuda là hai thư viện chính để chạy tối ưu hóa.

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
import time
import copy
import numpy as np
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

Chuẩn bị Dataloader (Test & Calibration)

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 1. Test Loader
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)

# 2. Calibration Loader
calib_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=test_transform)
calib_subset = torch.utils.data.Subset(calib_set, list(range(500)))
calib_loader = torch.utils.data.DataLoader(calib_subset, batch_size=1, shuffle=False)


Cài đặt các hàm tiện ích

In [ ]:
def get_model_size(model, label=""):
    """
    Hàm in kích thước mô hình bằng cách đọc file trọng số.
    :param model: Mô hình PyTorch.
    :param label: Nhãn tên cho mô hình.
    """
    torch.save(model.state_dict(), "temp.p")
    size = os.path.getsize("temp.p") / 1e6
    print(f"Model {label} size: {size:.2f} MB")
    os.remove("temp.p")
    return size

def evaluate_model(model, data_loader, device, max_samples = None, is_tensorrt = False):
    """Hàm đánh giá hỗ trợ cả PyTorch và TensorRT"""
    correct = 0
    total = 0
    start_time = time.time()

    if not is_tensorrt:
        model.eval()
        model.to(device)

    with torch.no_grad():
        for i, (images, labels) in enumerate(data_loader):
            if max_samples is not None and i >= max_samples:
                break

            labels = labels.to(device)

            if not is_tensorrt:
                images = images.to(device)
                outputs = model(images)
            else:
                outputs = model(images)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    end_time = time.time()
    accuracy = 100 * correct / total
    inference_time = (end_time - start_time) / total * 1000 # ms/img

    print(f" - Accuracy: {accuracy:.2f}%")
    print(f" - Running time ({min(total, max_samples)} samples): {inference_time:.4f} second")

    return accuracy, inference_time

def print_comparison(title, size_fp32, size_dyn, size_stat, time_fp32, time_dyn, time_stat, acc_fp32, acc_dyn, acc_stat):
    size_rate_dyn = size_fp32 / size_dyn
    size_rate_stat = size_fp32 / size_stat
    time_rate_dyn = time_fp32 / time_dyn
    time_rate_stat = time_fp32 / time_stat
    acc_delta_dyn = acc_fp32 - acc_dyn
    acc_delta_stat = acc_fp32 - acc_stat

    print("=" * 10 + f" {title} " + "=" * 10)
    print(f"{'Metric':<20} | {'FP32':<12} | {'INT8 Dyn':<12} | {'INT8 Stat':<12} | {'Cải thiện (Dyn/Stat)':<20}")
    print("-" * 95)
    print(f"{'Size (MB)':<20} | {size_fp32:>8.2f} MB | {size_dyn:>8.2f} MB  | {size_stat:>8.2f} MB  | Giảm {size_rate_dyn:.2f}x / {size_rate_stat:.2f}x")
    print(f"{'Accuracy (%)':<20} | {acc_fp32:>8.2f}%  | {acc_dyn:>8.2f}%   | {acc_stat:>8.2f}%   | Delta {acc_delta_dyn:+.2f}% / {acc_delta_stat:+.2f}%")
    print(f"{'Latency (s)':<20} | {time_fp32:>8.4f}s | {time_dyn:>8.4f}s | {time_stat:>8.4f}s | x{time_rate_dyn:.2f} / x{time_rate_stat:.2f} speedup")
    print("=" * 50)

Kiểm tra TensorRT

In [ ]:
import os
import sys

try:
    import tensorrt as trt
    print(f"TensorRT version: {trt.__version__}")
    logger = trt.Logger(trt.Logger.WARNING)
    builder = trt.Builder(logger)
    print("TensorRT Builder created successfully!")
except ImportError:
    print("TensorRT not found!")
except Exception as e:
    print(f"Error initializing TensorRT: {e}")

TensorRT version: 10.14.1.48.post1
TensorRT Builder created successfully! (CUDA driver matches)


 # **2. Quantize mô hình ResNet18 xuống FP16 và INT8 bằng TensorRT**

## 2.1. Khai báo mô hình, load trọng số và tính toán trên mô hình FP32

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_resnet18 = torchvision.models.resnet18(pretrained=False)
model_resnet18.fc = nn.Linear(512, 10)

weights_path = 'resnet18_cifar10_finetuned.pth'
if os.path.exists(weights_path):
    model_resnet18.load_state_dict(torch.load(weights_path, map_location=device))
else:
    print("Không tìm thấy file weights")

model_resnet18.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Lấy các thông số đánh giá của mô hình FP32.

In [ ]:
acc_fp32, time_fp32 = evaluate_model(model_resnet18, test_loader, device, max_samples = 500)

 - Accuracy: 93.00%
 - Running time (500 samples): 6.9641 second



## 2.2. TensorRT <br> **Step 1**: Chuyển đổi file trọng số của mô hình sang định dạng `ONNX`.
TensorRT không đọc trực tiếp file `.pth` của PyTorch. Chúng ta cần convert mô hình sang một định dạng khác là `ONNX`.

1. Đầu tiên tạo một dummpy_input, input này được truyền qua mô hình một lần, giúp `torch.onnx.export` truy vết được cấu trúc của mô hình.

In [ ]:
dummy_input = torch.randn(1, 3, 224, 224, device='cuda')

2. Dùng phương thức ` torch.onnx.export()` để chuyển đổi sang ONNX.


`export_params` =:

*  True: Lưu cả kiến trúc lẫn trọng số vào file ONNX.

*  False: Chỉ lưu khung mô hình

`opset_version `: ONNX có nhiều phiên bản tập lệnh (opset). Số 18 là một phiên bản khá mới. Chọn version cao giúp hỗ trợ nhiều toán tử phức tạp hơn, nhưng cần đảm bảo bản TensorRT bạn cài đặt hỗ trợ version này.

`do_constant_folding` = True: Tối ưu tính toán: Ví dụ: Nếu trong mạng có phép tính 2 * 3, thay vì lưu phép nhân, nó sẽ tính luôn ra 6 và lưu số 6 vào. Việc này giúp mô hình gọn nhẹ hơn.

`input_names = ['input']` & `output_names=['output']`: Gán nhãn cho cổng vào và cổng ra của file ONNX. Sau này khi viết code TensorRT chỉ cần gọi đúng tên `input` và `output` này để đẩy dữ liệu vào và lấy kết quả ra.

In [ ]:
onnx_file_path = "resnet18.onnx"

print(f"Exporting to ONNX: {onnx_file_path}...")

torch.onnx.export(
    model_resnet18,
    dummy_input,
    onnx_file_path,
    export_params=True,
    opset_version=18,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
)
print(" Export ONNX !")

Exporting to ONNX: resnet18.onnx...
[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 40 of general pattern rewrite rules.
 Export ONNX !


## 2.2. TensorRT <br> **Step 2**: Cài đặt quantization

Viết class hỗ trợ Calibration khi quantize về INT8 bằng TensorRT. Class này thừa kế `trt.IInt8EntropyCalibrator2`, là một thuật toán hỗ trợ sẵn để giảm thiếu mất mát khi nén dữ liệu.

In [ ]:
class ResNetEntropyCalibrator(trt.IInt8EntropyCalibrator2):
    def __init__(self, dataloader, cache_file="resnet18_calibration.cache"):
        super().__init__()
        self.dataloader = dataloader
        self.data_iter = iter(dataloader)
        self.cache_file = cache_file #Cache dữ liệu để tăng tốc độ
        self.batch_size = dataloader.batch_size
        self.current_batch = 0
        self.max_batches = 100 # Calibrate 100 batches
        self.device_input = cuda.mem_alloc(self.batch_size * 3 * 224 * 224 * 4) #Sử dụng pycuda để xin cấp phát một vùng nhớ trên GPU đủ chứa một batch ảnh đầu vào

    def get_batch_size(self): #
        """
        TensorRT gọi hàm này để biết kích thước batch size nạp vào
        """
        return self.batch_size

    def get_batch(self, names):
        """
        Quan trọng nhất.
        TensorRT sẽ gọi hàm này lặp đi lặp lại trong quá trình build engine để lấy dữ liệu cho đến khi hàm trả về None
        """
        if self.current_batch >= self.max_batches: return None
        try:
            data, _ = next(self.data_iter)
            batch_data = np.ascontiguousarray(data.numpy().astype(np.float32)) #Chuyển dữ liệu sang dạng mảng numpy liên tục trong bộ nhớ để copy cho an toàn
            cuda.memcpy_htod(self.device_input, batch_data) #Host to Device. Copy dữ liệu từ CPU sang vùng nhớ GPU (Device) mà ta đã cấp phát ở __init__.
            self.current_batch += 1
            return [int(self.device_input)] #Trả về địa chỉ bộ nhớ của dữ liệu trên GPU cho TensorRT xử lý.
        except StopIteration: return None

    def read_calibration_cache(self):
        """
        Đọc kết quả hiệu chuẩn.
        """
        return open(self.cache_file, "rb").read() if os.path.exists(self.cache_file) else None

    def write_calibration_cache(self, cache):
        """
        Lưu kết quả hiệu chuẩn vào file cache.
        """
        open(self.cache_file, "wb").write(cache)

In [ ]:
def build_engine(onnx_path, engine_path, mode='fp16', dataloader=None):
    logger = trt.Logger(trt.Logger.WARNING) #Ghi log
    builder = trt.Builder(logger) #builder có nhiệm vụ xây engine
    network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)) #Tạo nơi chứa định nghĩa mô hình, cờ EXPLICIT_BATCH là bắt buộc với ONNX.
    parser = trt.OnnxParser(network, logger) #Đọc file onnx
    config = builder.create_builder_config()

    with open(onnx_path, 'rb') as model: #Đọc file onnx
        parser.parse(model.read())

    config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30) # Vùng nhớ tạm thời TensorRT dùng để thử nghiệm các thuật toán tối ưu, 2^30 = 1GB

    if mode == 'fp16' and builder.platform_has_fast_fp16: #Lựa chọn tối ưu về fp16
        config.set_flag(trt.BuilderFlag.FP16) #Gắn cờ vào config()
    elif mode == 'int8' and builder.platform_has_fast_int8: #Lựa chọn tối ưu về int8
        config.set_flag(trt.BuilderFlag.INT8)
        config.int8_calibrator = ResNetEntropyCalibrator(dataloader, f"calib_{mode}.cache")

    print(f"Building {mode} engine...")
    serialized_engine = builder.build_serialized_network(network, config) # Bắt đầu nén
    with open(engine_path, "wb") as f:
        f.write(serialized_engine)
    print(f"✅ Saved: {engine_path}")

In [ ]:
build_engine(onnx_file_path, "resnet18_fp16.trt", mode='fp16')
build_engine(onnx_file_path, "resnet18_int8.trt", mode='int8', dataloader=calib_loader)

Building fp16 engine...
✅ Saved: resnet18_fp16.trt
Building int8 engine...


/tmp/ipython-input-3659170000.py:17: DeprecationWarning: Use Deprecated in TensorRT 10.1. Superseded by explicit quantization. instead.
  config.int8_calibrator = ResNetEntropyCalibrator(dataloader, f"calib_{mode}.cache")


✅ Saved: resnet18_int8.trt


Class `TRTModuleWrapper` thừa kế từ `nn.Module` của PyTorch. Mục đích là để đóng giả một layer PyTorch, nhưng bên trong lại chạy bằng động cơ TensorRT. Nhờ vậy, bạn có thể tái sử dụng toàn bộ code đánh giá (evaluate_model) cũ mà không cần sửa đổi gì.

In [ ]:
class TRTModuleWrapper(nn.Module):
    def __init__(self, engine_path):
        super().__init__()
        self.logger = trt.Logger(trt.Logger.WARNING)
        self.runtime = trt.Runtime(self.logger) #trt.Runtime để đọc file engine.

        with open(engine_path, "rb") as f:
            self.engine = self.runtime.deserialize_cuda_engine(f.read()) #Đọc file nhị phân .trt từ ổ cứng và giải nén nó thành đối tượng self.engine để sẵn sàng sử dụng.

        self.context = self.engine.create_execution_context() #môi trường thực thi

        # TensorRT yêu cầu bạn phải tự quản lý bộ nhớ thủ công.
        # Vòng lặp for sau sẽ thực hiện:
            # Duyệt qua từng đầu vào/đầu ra của mô hình để xem nó cần bao nhiêu bytes.
            # Sử dụng cuda.pagelocked_empty để copy dữ liệu sang GPU nhanh gấp đôi so với bộ nhớ RAM thường.
            # Sử dụng cuda.mem_alloc để xin VRAM trên GPU.
            # Lưu lại địa chỉ của các vùng nhớ này để chỉ cho TensorRT biết chỗ nào chứa ảnh, chỗ nào chứa kết quả
        self.inputs = []
        self.outputs = []
        self.allocations = []
        self.stream = cuda.Stream()

        for i in range(self.engine.num_io_tensors):
            name = self.engine.get_tensor_name(i)
            # Lấy shape và dtype
            shape = self.engine.get_tensor_shape(name)
            dtype = self.engine.get_tensor_dtype(name)

            # Tính toán kích thước bộ nhớ cần thiết
            size = trt.volume(shape)
            numpy_dtype = trt.nptype(dtype)

            # Cấp phát Host (CPU) và Device (GPU) memory
            host_mem = cuda.pagelocked_empty(size, numpy_dtype)
            device_mem = cuda.mem_alloc(host_mem.nbytes)

            self.allocations.append(int(device_mem))
            self.context.set_tensor_address(name, int(device_mem))

            # Lưu binding info
            # Lưu shape dưới dạng tuple ngay từ đây để dùng về sau
            binding = {
                'name': name,
                'host': host_mem,
                'device': device_mem,
                'shape': tuple(shape)
            }

            if self.engine.get_tensor_mode(name) == trt.TensorIOMode.INPUT:
                self.inputs.append(binding)
            else:
                self.outputs.append(binding)

    def forward(self, x):
        # x là PyTorch Tensor (Batch, 3, 224, 224)
        # 1. Copy Tensor -> Numpy -> Host Buffer
        input_np = x.cpu().numpy().ravel()
        np.copyto(self.inputs[0]['host'], input_np)

        # 2. Host -> Device
        cuda.memcpy_htod_async(self.inputs[0]['device'], self.inputs[0]['host'], self.stream)

        # 3. Infer
        self.context.execute_async_v3(stream_handle=self.stream.handle)

        # 4. Device -> Host
        cuda.memcpy_dtoh_async(self.outputs[0]['host'], self.outputs[0]['device'], self.stream)
        self.stream.synchronize()

        # 5. Return Tensor (Batch, Classes)
        output_data = self.outputs[0]['host']

        # Lấy shape đã được convert sang tuple ở hàm __init__
        output_shape = self.outputs[0]['shape']

        return torch.from_numpy(output_data).reshape(output_shape).to(x.device)


**Bảng so sánh**

In [ ]:
import os
def get_file_size_mb(file_path):
    if os.path.exists(file_path):
        return os.path.getsize(file_path) / 1e6
    return 0

# Kiểm tra file tồn tại trước khi chạy
if os.path.exists("resnet18_fp16.trt") and os.path.exists("resnet18_int8.trt"):
    # Load Engines
    trt_fp16 = TRTModuleWrapper("resnet18_fp16.trt")
    trt_int8 = TRTModuleWrapper("resnet18_int8.trt")

    print("Evaluating TRT FP16...")
    acc_fp16, lat_fp16 = evaluate_model(trt_fp16, test_loader, device, max_samples=500, is_tensorrt=True)
    size_fp16 = get_file_size_mb("resnet18_fp16.trt")

    print("Evaluating TRT INT8...")
    acc_int8, lat_int8 = evaluate_model(trt_int8, test_loader, device, max_samples=500, is_tensorrt=True)
    size_int8 = get_file_size_mb("resnet18_int8.trt")

    # 3. Tổng hợp kết quả
    size_pt = get_file_size_mb(weights_path) if os.path.exists(weights_path) else 0

    print(f"\n{'Method':<15} | {'Size (MB)':<10} | {'Acc (%)':<10} | {'Time (ms)':<15} | {'Speedup':<10}")
    print("-" * 75)
    print(f"{'PyTorch FP32':<15} | {size_pt:<10.2f} | {acc_pt:<10.2f} | {lat_pt:<15.2f} | {'1.0x':<10}")
    print(f"{'TRT FP16':<15} | {size_fp16:<10.2f} | {acc_fp16:<10.2f} | {lat_fp16:<15.2f} | {lat_pt/lat_fp16:<10.2f}")
    print(f"{'TRT INT8':<15} | {size_int8:<10.2f} | {acc_int8:<10.2f} | {lat_int8:<15.2f} | {lat_pt/lat_int8:<10.2f}")

else:
    print("Error: Không tìm thấy file .trt!")

Evaluating TRT FP16...
 - Accuracy: 93.00%
 - Running time (500 samples): 4.6443 second
Evaluating TRT INT8...
 - Accuracy: 92.80%
 - Running time (500 samples): 3.9797 second

Method          | Size (MB)  | Acc (%)    | Time (ms)       | Speedup   
---------------------------------------------------------------------------
PyTorch FP32    | 44.81      | 93.00      | 6.96            | 1.0x      
TRT FP16        | 22.61      | 93.00      | 4.64            | 1.50      
TRT INT8        | 11.79      | 92.80      | 3.98            | 1.75      
